# Week 1

### STEP 1: Data Import

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility_20260120.csv')


Mounted at /content/drive


/tmp/ipython-input-2834733236.py:5: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/My Drive/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility_20260120.csv')


In [2]:
df

,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_coverage,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_coverage,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_coverage,total_staffed_pediatric_icu_beds_7_day_sum
0,370094,2021/01/10,OK,370094,SSM HEALTH ST ANTHONY HOSPITAL - MIDWEST,2825 PARKLAWN DRIVE,MIDWEST CITY,73110.0,Short Term,40109.0,...,NaN,NaN,0,NaN,0,3,0,0,3,0
1,370220,2020/06/14,OK,370220,ONECORE HEALTH,100 NE 85TH STREET,OKLAHOMA CITY,73114.0,Short Term,40109.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
2,431338,2022/12/25,SD,431338,AVERA GREGORY HOSPITAL,110 S LOGAN AVE,GREGORY,57533.0,Critical Access Hospitals,46053.0,...,0,0,7,0,0,7,0,0,7,0
3,450424,2020/10/18,TX,450424,HOUSTON METHODIST BAYTOWN HOSPITAL,4401 GARTH ROAD,BAYTOWN,77521.0,Short Term,48201.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
4,453316,2020/08/02,TX,453316,CHILDRENS MEDICAL CENTER PLANO,7601 PRESTON ROAD,PLANO,75024.0,Childrens Hospitals,48085.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045401,420030,2021/10/03,SC,420030,COLLETON MEDICAL CENTER,501 ROBERTSON BOULEVARD,WALTERBORO,29488.0,Short Term,45029.0,...,NaN,NaN,0,NaN,0,7,0,0,7,0
1045402,431302,2022/01/23,SD,431302,AVERA MISSOURI RIVER HEALTH CENTER,606 EAST GARFIELD,GETTYSBURG,57442.0,Critical Access Hospitals,46107.0,...,0,"-999,999",3,"-999,999",0,5,0,0,5,0
1045403,450578,2021/09/12,TX,450578,HEMPHILL COUNTY HOSPITAL,1020 S 4TH ST,CANADIAN,79014.0,Short Term,48211.0,...,NaN,NaN,0,NaN,0,7,0,0,7,0
1045404,420067,2021/04/25,SC,420067,BEAUFORT COUNTY MEMORIAL HOSPITAL,955 RIBAUT RD,BEAUFORT,29902.0,Short Term,45013.0,...,NaN,NaN,0,NaN,0,7,0,0,7,0


### STEP 2: Data Preprocessing and Descriptive Analysis

In [3]:
import pandas as pd

# load dataset
df = pd.read_csv(
    '/content/drive/My Drive/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility_20260120.csv',
    dtype={'hospital_pk': str, 'ccn': str},
    low_memory=False
)

# converting date
df['collection_week'] = pd.to_datetime(df['collection_week'], errors='coerce')

df.replace(-999999, pd.NA, inplace=True)

# columns
key_metrics = [
    'previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum',
    'staffed_icu_pediatric_patients_confirmed_covid_7_day_avg',
    'staffed_pediatric_icu_bed_occupancy_7_day_avg',
    'total_staffed_pediatric_icu_beds_7_day_avg'
]

for col in key_metrics:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col] = df[col].fillna(df[col].mean())

# duplicates
df = df.drop_duplicates(subset=['hospital_pk', 'collection_week'])

summary_stats = df[key_metrics].describe()
print(summary_stats)


       previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum  \
count                                       1.045406e+06                    
mean                                        2.547349e-02                    
std                                         6.243036e-01                    
min                                        -5.000000e+00                    
25%                                         0.000000e+00                    
50%                                         0.000000e+00                    
75%                                         2.547349e-02                    
max                                         2.520000e+02                    

       staffed_icu_pediatric_patients_confirmed_covid_7_day_avg  \
count                                       1.045406e+06          
mean                                        7.702848e-03          
std                                         2.146865e-01          
min                                   

### STEP 3: Missing Values and Data Validation

In [4]:
for col in key_metrics:
    df[col] = df[col].apply(lambda x: x if x >= 0 else 0)


In [5]:
# missing values per column
missing_counts = df.isna().sum()
print(missing_counts[missing_counts > 0])  # only show columns with missing data


ccn                                                                   1402
address                                                                442
city                                                                   442
zip                                                                     19
fips_code                                                              986
                                                                     ...  
previous_day_admission_pediatric_covid_confirmed_12_17_7_day_sum    489120
previous_day_admission_pediatric_covid_confirmed_5_11_7_day_sum     487573
staffed_icu_pediatric_patients_confirmed_covid_7_day_sum            417886
staffed_pediatric_icu_bed_occupancy_7_day_sum                       199396
total_staffed_pediatric_icu_beds_7_day_sum                          201131
Length: 87, dtype: int64


In [6]:
df = df.dropna(subset=['hospital_pk'])


In [7]:
pediatric_cols = [
    'previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum',
    'staffed_icu_pediatric_patients_confirmed_covid_7_day_avg',
    'staffed_pediatric_icu_bed_occupancy_7_day_avg',
    'total_staffed_pediatric_icu_beds_7_day_avg'
]

for col in pediatric_cols:
    df[col] = df[col].fillna(0)



In [8]:
print(df[pediatric_cols].isna().sum())


previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum    0
staffed_icu_pediatric_patients_confirmed_covid_7_day_avg              0
staffed_pediatric_icu_bed_occupancy_7_day_avg                         0
total_staffed_pediatric_icu_beds_7_day_avg                            0
dtype: int64


### STEP 4: Outlier Detection

In [9]:
# only numeric dtype columns
numeric_cols_clean = df.select_dtypes(include=['number']).columns

# fill missing and remove negatives
for col in numeric_cols_clean:
    df[col] = df[col].fillna(0)
    df[col] = df[col].clip(lower=0)

outlier_report = {}
for col in numeric_cols_clean:
    threshold = df[col].quantile(0.99)
    outlier_report[col] = (df[col] > threshold).sum()

metadata_cols = ['hospital_pk', 'ccn', 'hospital_name', 'address', 'city',
                 'zip', 'state', 'hospital_subtype', 'fips_code', 'collection_week']

missing_metadata = df[metadata_cols].isna().sum()

print("Dataset Cleaning Report\n")
print("Missing values in metadata columns:")
print(missing_metadata[missing_metadata > 0] if missing_metadata.any() else "None")

print("\nNumber of outliers (above 99th percentile) per numeric column:")
for col, count in outlier_report.items():
    if count > 0:
        print(f"{col}: {count}")

print("\nTotal rows after cleaning:", len(df))
print("All numeric columns are now >= 0 and have no missing values.")


Dataset Cleaning Report

Missing values in metadata columns:
ccn        1402
address     442
city        442
dtype: int64

Number of outliers (above 99th percentile) per numeric column:
zip: 10258
fips_code: 10431
previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum: 1291
staffed_icu_pediatric_patients_confirmed_covid_7_day_avg: 684
staffed_pediatric_icu_bed_occupancy_7_day_avg: 10427
total_staffed_pediatric_icu_beds_7_day_avg: 10442

Total rows after cleaning: 1045406
All numeric columns are now >= 0 and have no missing values.


# Week 2

### STEP 5. ICU Stress Indicator Creation

In [11]:
df['pediatric_icu_utilization_rate'] = (
    df['staffed_pediatric_icu_bed_occupancy_7_day_avg'] /
    df['total_staffed_pediatric_icu_beds_7_day_avg']
)

# replace infinite values
df['pediatric_icu_utilization_rate'] = df['pediatric_icu_utilization_rate'].replace([float('inf')], 0)

print(df['pediatric_icu_utilization_rate'].describe())

count    313379.000000
mean          0.622834
std           0.337763
min           0.000000
25%           0.665608
50%           0.665608
75%           0.665608
max          54.455094
Name: pediatric_icu_utilization_rate, dtype: float64


### STEP 6. High-Stress Hospital Identification

In [12]:
# stress threshold
high_stress = df[df['pediatric_icu_utilization_rate'] > 0.8]

print("Number of high-stress hospital-week records:", len(high_stress))

Number of high-stress hospital-week records: 26464


### STEP 7. Correlation Analysis

In [18]:
# columns for correlation
corr_cols = [
    'previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum',
    'staffed_icu_pediatric_patients_confirmed_covid_7_day_avg',
    'total_staffed_pediatric_icu_beds_7_day_avg',
    'total_patients_hospitalized_confirmed_influenza_7_day_avg',
    'pediatric_icu_utilization_rate'
]

# removing commas
for col in corr_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(',', '', regex=False)
    )
    df[col] = pd.to_numeric(df[col], errors='coerce')

df[corr_cols] = df[corr_cols].replace(-999999, 0)



In [16]:
correlation_matrix = df[corr_cols].corr()
print(correlation_matrix)

                                                    previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum  \
previous_day_admission_pediatric_covid_confirme...                                           1.000000                    
staffed_icu_pediatric_patients_confirmed_covid_...                                           0.046607                    
total_staffed_pediatric_icu_beds_7_day_avg                                                   0.039563                    
total_patients_hospitalized_confirmed_influenza...                                           0.007087                    
pediatric_icu_utilization_rate                                                               0.035122                    

                                                    staffed_icu_pediatric_patients_confirmed_covid_7_day_avg  \
previous_day_admission_pediatric_covid_confirme...                                           0.046607          
staffed_icu_pediatric_patients_confirmed_co

### STEP 8. Hospital-Level Variability

In [14]:
hospital_variability = df.groupby('hospital_pk')[
    'pediatric_icu_utilization_rate'
].std().sort_values(ascending=False)

print("Top 10 Most Variable Hospitals:")
print(hospital_variability.head(10))

Top 10 Most Variable Hospitals:
hospital_pk
260032    10.475824
330226     5.830787
500054     3.635138
410010     2.657537
500014     2.328674
050056     2.155234
053305     2.121323
020001     2.046637
030087     1.880654
033302     1.795083
Name: pediatric_icu_utilization_rate, dtype: float64
